In [ ]:
!pip install --user py4j
!pip install --user pyspark

In [1]:

import pyspark
from pyspark.sql import SQLContext
from pyspark.sql import SparkSession


In [2]:

#.config("hive.metastore.uris", "thrift://172.17.0.1:9083") \
#.config("spark.driver.allowMultipleContexts",  True) \
#.config("spark.hadoop.fs.defaultFS", "hdfs://172.17.0.1:8020") \
#.config("spark.sql.warehouse.dir", "/user/hive/warehouse") \

spark = SparkSession \
    .builder \
    .appName("dsx-test") \
    .config("spark.driver.allowMultipleContexts",  True) \
    .config("spark.hadoop.fs.defaultFS", "webhdfs://192.168.54.10:50070") \
    .config("hive.metastore.uris", "thrift://172.17.0.1:9083") \
    .enableHiveSupport() \
    .getOrCreate()


IllegalArgumentException: "Error while instantiating 'org.apache.spark.sql.hive.HiveSessionStateBuilder':"

In [ ]:

tables = spark.sql("show tables")
tables.show()


In [ ]:

customers = spark.sql("describe table customers")
customers.show()


In [ ]:

customers = spark.sql("select name from customers where name = 'toto'")
customers.show()


In [ ]:

#import os
#os.environ['HTTP_PROXY'] = "http://172.17.0.1:3080"

import pandas as pd

customers = pd.read_csv('../datasets/customer_details_noquotes.csv', dtype='object')
#customers.info()

from pyspark.sql.types import *
schema = StructType([
    StructField("name", StringType(), True),
    StructField("subscription_date", StringType(), True),
    StructField("phone_brand", StringType(), True),
    StructField("phone_model", StringType(), True),
    StructField("phone", StringType(), True),
    StructField("state", StringType(), True),
    StructField("account_length", StringType(), True),
    StructField("international_plan", StringType(), True),
    StructField("voice_mail_plan", StringType(), True),
    StructField("number_vmail_messages", StringType(), True),
    StructField("customer_service_calls", StringType(), True),
    StructField("churn", StringType(), True)])

customers = spark.createDataFrame(customers, schema)

#customers = spark.read.csv('file:///../datasets/customer_details.csv', header='true', inferSchema = 'true')
#customers = spark.read.csv('webhdfs://192.168.54.10:50070/user/bruno.bonnin/customer_details.csv?user.bonnin=bruno.bonnin', headerr='true', inferSchema = 'true')
#customers = spark.read.csv('hdfs://172.17.0.1:8020/user/bruno.bonnin/customer_details.csv', header='true', inferSchema = 'true')
#customers = spark.read.csv('/user/bruno.bonnin/customer_details.csv', header='true', inferSchema = 'true')
customers_apple = customers.filter(customers.phone_brand == 'Apple')
customers_apple.show(3)



In [ ]:

customers_apple.write.saveAsTable('telco_customers_apple')


In [ ]:
from pyspark.sql.functions import countDistinct
by_states = customers.groupBy('state').agg(countDistinct('phone'))
by_states.show()

In [ ]:

import pyspark
from pyspark.sql import SQLContext

if 'sc' in locals() or 'sc' in globals():
    sc.stop()

conf = pyspark.SparkConf()
conf.set("spark.driver.allowMultipleContexts",  True)
conf.set("spark.driver.extraClassPath",  "/user-home/999/.local/lib/python3.5/site-packages/pyspark/rjdbc-hive_1.1.1-1.0-jar-with-dependencies.jar")
sc = pyspark.SparkContext(appName = "test-dsx", conf = conf)
sqlContext = SQLContext(sc)

#dbtable="(select name, phone_brand, state from customer_details) phone_state",
customer_details = sqlContext.read \
    .format("jdbc") \
    .options(driver="org.apache.hive.jdbc.HiveDriver", 
             url="jdbc:hive2://172.17.0.1:10000/churn", 
             dbtable="(select name, phone, state from customer_details) p",
             fetchsize=20,
             user="bruno.bonnin", 
             password="Matlol1998") \
    .load()

customer_details.printSchema()



In [ ]:

customer_details.show(5)


In [ ]:
customers_apple = customer_details.filter("`phone_state.phone_brand` = 'Apple'")
customers_apple.show(3)